<a href="https://colab.research.google.com/github/VS-Coder/DS-Unit-2-Linear-Models/blob/master/Michael_Davis_LS_DS_212_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [2]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

In [4]:
# Converting the string datetime values into a DateTime value.
df['created'] = pd.to_datetime(df['created'], infer_datetime_format= True)

In [5]:
# Mapping the interest_level to numeric values from string values to use as a feature.
# df['interest_level'] = df['interest_level'].map({'low': 1, 'medium': 2, 'high': 3})

In [6]:
# April and May 2016 data. Train data.
train = df[(df['created'] >= '2016-04-01') & (df['created'] < '2016-06-01')]

In [7]:
# June Data. Test data.
test = df[(df['created'] >= '2016-06-01')]

In [53]:
# Creating the features and the target variables.
# The features are bathrooms, bedrooms, and the location information.
# The target variable is the price.
feats = ['bathrooms', 'bedrooms', 'latitude', 'longitude']
targ = ['price']

In [63]:
# More imports.
import plotly.express as px 
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

In [10]:
# Instantiating the LinearRegression class.
model = LinearRegression()

In [54]:
# Subsetting the dataframe for the training values that will 
# be used for making the preditions.
x_train = train[feats]
y_train = train[targ]

x_test = test[feats]
y_test = test[targ]

In [55]:
model.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [62]:
# Defining the function to be used for caluculations of the estimate and
# coefficient from the data model.
def pred(x_tst, y_true):
  ''' Function pred(x_tst, y_true)
          Predicts the rent value based on the test data passed into the Linear Regression
          model.

  Returns y_pred - array
            The prediction of the rent value.
          coeff - array
            The coefficient value based on the test data.
          mod_int - array
            The model intercept value based on the training data passed into
            the Linear Regression model.
          r2_scr - float
            The R^2 score for the test data, and the prediction.'''

  # y_pred is the prediction being made based on the test data passed into
  # the function.
  y_pred = model.predict(x_tst)
  # coeff is the coefficient value based on the model that is being used in this function.
  coeff = model.coef_
  # mod_int is the model intercept value.
  mod_int = model.intercept_
  # R^2 score
  r2_scr = r2_score(y_true, y_pred)
  # The returned values are the estimate and coefficient.
  return y_pred, coeff, mod_int, r2_scr

In [60]:
# Calling the function and passing the x_test and y_test dataframes.
pred_vals = pred(x_test, y_test)
pred_vals

(array([[4236.10857798],
        [3350.89266327],
        [2573.32292805],
        ...,
        [3278.834954  ],
        [3414.80459907],
        [3454.62232743]]),
 array([[  2002.79660997,    428.45152883,   1863.91519176,
         -16328.42647587]]),
 array([-1283306.49069029]),
 0.5882903576478155)

In [61]:
# Verifying that the prediction data and test data are the same shape.
print(pred_vals[0].shape, y_test.shape)

(16973, 1) (16973, 1)


In [58]:
# Calulating the mean absolute error value
mae = mean_absolute_error(y_test, pred_vals[0])
mae

744.975265812059

In [65]:
# Calulating the Root Mean Squared Error value.
rmse = mean_squared_error(y_test, pred_vals[0], squared= False)
rmse

1131.1950868928673